In [1]:
from cmless.parse import parse_cmless, parse_dir
from rich import print

In [2]:
from cmless.models import Collection, Exhibit
# c = parse_cmless('app/views/special_collections/zoom.md')
# parse_cmless('app/views/special_collections/radio-kdna.md')
parse_cmless('app/views/special_collections/rock-and-roll.md')
# c = Collection(Title='Test', Slug='asdf', Summary='asdf')

{'Title': 'Rock & Roll',
 'Slug': 'rock-and-roll',
 'Thumbnail': '![Rock & Roll](https://s3.amazonaws.com/americanarchive.org/special-collections/rock.jpg "Rock & Roll")',
 'Summary': 'The <em>Rock & Roll</em> Special Collection features 166 raw interviews from the 10-part series of the same name, also known in the UK as <em>Dancing in the Street: a Rock and Roll History</em>. The series, which focused on the progression of rock music from its beginnings in the 1950s to its contemporary forms in the 1990s, was co-produced by WGBH and the BBC. The series used interviews with artists and industry insiders to examine the influence and growth of musical styles like R&B, jazz, soul, heavy metal, glam, reggae, country, and rap on the rock and roll scene. The series, narrated by Emmy- and Golden Globe-nominated actor Liev Schreiber, was the recipient of a Peabody Award. Interviews in the collection include conversations with musicians, record producers, recording engineers, and consultants, i

In [3]:
collections = parse_dir('app/views/special_collections')
len(collections)
# print(collections[-1])

96

In [4]:
from os import listdir, path

exhibits = {}

# Parse all the exhibits in the app/views/exhibits directory
for exhibit in listdir('app/views/exhibits'):
    # print(exhibit)
    exhibit_path = path.join('app/views/exhibits', exhibit)
    slug = exhibit.replace('.md', '')
    if exhibit.endswith('.md'):
        exhibit = parse_cmless(exhibit_path)
        exhibit = Exhibit(**exhibit)
        exhibits[slug] = exhibit
    if path.isdir(exhibit_path):
        exhibits[slug] = {}
        for exhibit_file in listdir(exhibit_path):
            if exhibit_file.endswith('.md'):
                exhibit = parse_cmless(path.join(exhibit_path, exhibit_file))
                page_slug = exhibit_file.replace('.md', '')
                exhibit = Exhibit(**exhibit,)
                exhibits[slug][page_slug] = exhibit

In [6]:
# print(exhibits)
len(exhibits)
# print(exhibits['peabody'])

25

In [ ]:
parse_cmless('app/views/exhibits/zoom.md')

In [ ]:
import re

def cmless2(file):
    with open(file, 'r') as f:
        md = f.read()
    title_pattern = r"^#\s+(.*)"
    section_pattern = r"^##\s+(.*)"
    titles = re.split(title_pattern, md)
    assert titles[0].strip() == '', "There should be nothing before the first title"
    assert len(titles) == 3, "There should be exactly 3 parts to this match"
    title = titles[1].strip()
    body = titles[2].strip()

    
    sections = re.split(section_pattern, body, flags=re.MULTILINE)
    try:
        # If the first section is a number, it means we have a page number
        page = int(sections[0].strip())
    except ValueError:
        # Nevermind
        page = None
        assert sections[0].strip() == '', "First section should be empty if there's no page number"
            
    # Remove the empty first section
    sections = sections[1:]

    # Turn the remaining sections into a dictionary
    sections = {sections[i].strip(): sections[i + 1].strip() for i in range(0, len(sections), 2)}


    sections = {k: v for k, v in sections.items() if k and v}

    result = {
        'Title': title,
        # 'body': body,
        **sections,
    }
    if page is not None:
        result['Page'] = page
    return result

cmless2('app/views/exhibits/zoom.md')